In [1]:
import warnings
warnings.filterwarnings('ignore')
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from leven import jamo_levenshtein
import pandas as pd

In [8]:
data = pd.read_excel('../00.참고자료/스마트건설_Rev.01.xlsx', names=['name','size','amount','unit','price','notes'])
data = data.iloc[1:]
data = data.dropna(subset=['price'], how='any').reset_index(drop=True)
data.iloc[26:]
data1 = data.iloc[0:4]
data2 = data.iloc[5:14]
data3 = data.iloc[15:27]
data4 = data.iloc[28:38]
data4 = data.iloc[39:49]
data4 = data.iloc[50:57]


In [9]:
data.head(100)

,name,size,amount,unit,price,notes
0,...a-1 땅깎기(토사),신설(도쟈32ton),1,㎥,720,도로공사
1,a. 토사깍기,도쟈32TON,1,㎥,720,조달청
2,흙깎기공,"토사,대규모(실적)",1,㎥,720,철도시설공단
3,(1). 흙깎기공/토사,NaN,1,㎥,720,LH공사
4,공 종,규 격,수 량,단 위,단 가,비 고
5,터파기,백호0.4㎥(100%),1,m3,1800,관로터파기
6,...터파기:토사(육상),B.H 0.4㎥,1,M3,1800,NaN
7,..터파기(토사 기계 100%),B.H0.4M3(육상),1,M3,1800,NaN
8,터파기,"백호 0.4㎥,기계 100%",1,㎥,1800,NaN
9,"터파기(토사,육상)",B.H 0.4㎥+크람쉘,1,M3,3000,터파기+크람쉘


In [10]:
data1

,name,size,amount,unit,price,notes
0,...a-1 땅깎기(토사),신설(도쟈32ton),1,㎥,720,도로공사
1,a. 토사깍기,도쟈32TON,1,㎥,720,조달청
2,흙깎기공,"토사,대규모(실적)",1,㎥,720,철도시설공단
3,(1). 흙깎기공/토사,NaN,1,㎥,720,LH공사


In [11]:
# import re
# def cleanText(data):
#     #텍스트에 포함되어 있는 특수 문자 제거
#     text= re.sub('[-=+,#/\?:.a-zA-Z^%$*@\'\(\(\[\]\<\>\(\)\s]','', data)
#     return text

In [12]:
import re
def cleanText(data):
    #텍스트에 포함되어 있는 특수 문자 제거
    text= re.sub('[-=+,#/\?:.^%$*@\'\(\(\[\]\<\>\(\)\s]','', data)
    return text

In [13]:
from konlpy.tag import Okt  
okt=Okt()  
token_result = []
for i in data['name']:
    token_result.append(okt.morphs(cleanText(i)))

In [14]:
okt.morphs(cleanText('...터파기:토사(육상)	B.H 0.4㎥'))

['터', '파기', '토사', '육상', 'BH', '04', '㎥']

In [15]:
text2 = '터파기 / 백호0.4㎥(100%)'

In [16]:
okt.morphs(cleanText(text2))

['터', '파기', '백호', '04', '㎥', '100']

In [17]:
from konlpy.tag import Kkma  
kkma=Kkma()  
token_result1 = []
for i in data['name']:
    token_result1.append(kkma.morphs(cleanText(i)))

In [18]:
token_result

[['a', '1', '땅', '깎기', '토사'],
 ['a', '토사', '깍', '기'],
 ['흙', '깎', '기공'],
 ['1', '흙', '깎기', '공', '토사'],
 ['공종'],
 ['터', '파기'],
 ['터', '파기', '토사', '육상'],
 ['터', '파기', '토사', '기계', '100'],
 ['터', '파기'],
 ['터', '파기', '토사', '육상'],
 ['터', '파기', '육상', '토사', 'BH', '04', '㎥'],
 ['터', '파기', '토사'],
 ['터', '파기', '토사'],
 ['터', '파기', '토사'],
 ['공종'],
 ['b', '합판', '거푸집'],
 ['a', '3', '거푸집', '거친', '마감'],
 ['합판', '거푸집'],
 ['c', '합판', '거푸집'],
 ['b', '합판', '거푸집'],
 ['a', '3', '거푸집', '보통', '마감'],
 ['합판', '거푸집'],
 ['c', '합판', '거푸집'],
 ['b', '합판', '거푸집'],
 ['a', '3', '거푸집', '매끈한', '마감'],
 ['합판', '거푸집'],
 ['c', '합판', '거푸집'],
 ['공종'],
 ['a', '1', '철근콘크리트', '타', '설', '펌프', '카', '실적'],
 ['a', '2', '철근콘크리트', 'H', '15', 'm', '미만'],
 ['a', '콘크리트', '타', '설'],
 ['1', '철근콘크리트', '타', '설'],
 ['b', '콘크리트', '타', '설', '펌프', '카'],
 ['1', '철근콘크리트', '타', '설'],
 ['1', '콘크리트', '타', '설'],
 ['c', '21', '철근콘크리트', '타', '설', '펌프', '카', '표준', '시장'],
 ['b', '1', '콘크리트', '타', '설', '철근'],
 ['a', '1', '철근콘크리트', '타', '설'],
 ['공종'],
 ['2', 

In [19]:
final = []
for i in token_result:
    for j in i:
        final.append(j)

In [21]:
final = set(final)

In [22]:
final

{'04',
 '1',
 '100',
 '12',
 '15',
 '1천',
 '2',
 '21',
 '23',
 '3',
 '5',
 '5080',
 '6096',
 '8',
 'BH',
 'D',
 'H',
 'MM',
 'a',
 'b',
 'c',
 'e',
 'f',
 'g',
 'h',
 'm',
 'n',
 'p',
 's',
 '㎥',
 '가유',
 '거친',
 '거푸집',
 '고',
 '공',
 '공관',
 '공종',
 '기',
 '기계',
 '기공',
 '깍',
 '깎',
 '깎기',
 '두부',
 '땅',
 '마감',
 '매끈한',
 '물',
 '미만',
 '밀도파',
 '보강',
 '보통',
 '설',
 '설치',
 '수축',
 '시장',
 '실란트',
 '실란트충진',
 '실런트',
 '실적',
 '앵커',
 '어스',
 '용',
 '웨더',
 '유',
 '육상',
 '줄눈',
 '철근',
 '철근콘크리트',
 '카',
 '콘크리트',
 '타',
 '터',
 '터널',
 '토사',
 '파기',
 '펌프',
 '폴리에틸렌',
 '표준',
 '푸',
 '푸기',
 '합판',
 '형',
 '흙'}

In [5]:
final = list(final)

NameError: name 'final' is not defined

In [6]:
for i in final:
    print(i+',', end='')

NameError: name 'final' is not defined

In [7]:
final_df = pd.DataFrame(final)

NameError: name 'final' is not defined

In [63]:
#final_df.to_csv('../03.Implementation/0701final_csv.csv', index=False, encoding='cp949')

In [67]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(final)

In [68]:
print(t.word_index)

{'철근콘크리트': 1, '거푸집': 2, '매끈한': 3, '공': 4, '기': 5, '육상': 6, '어스': 7, '실란트충진': 8, '유': 9, '가유': 10, '깍': 11, '기공': 12, '표준': 13, '웨더': 14, '철근': 15, '1': 16, '3': 17, '푸기': 18, '콘크리트': 19, '1천': 20, '마감': 21, '미만': 22, '04': 23, '공관': 24, '파기': 25, '거친': 26, '126096': 27, '카': 28, '실런트': 29, '앵커': 30, '실란트': 31, '깎기': 32, '흙': 33, '합판': 34, '공종': 35, '타': 36, '두부': 37, '25080': 38, '21': 39, '설': 40, '236096': 41, '실적': 42, '시장': 43, '땅': 44, '폴리에틸렌': 45, '수축': 46, '터': 47, '깎': 48, '형': 49, '물': 50, '보강': 51, '펌프': 52, '기계': 53, '8': 54, '2': 55, '터널': 56, '토사': 57, '밀도파': 58, '보통': 59, '100': 60, '15': 61, '줄눈': 62, '푸': 63, '5': 64, '용': 65, '설치': 66, '고': 67, '㎥': 68}


In [69]:
x=t.texts_to_sequences(final)
print(x)

[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68]]


In [70]:
vocab_size = len(t.word_index) # 단어 집합의 크기. 이 경우는 단어의 개수가 7이므로 7.

from keras.utils import to_categorical
x = to_categorical(x, num_classes=vocab_size+1) # 실제 단어 집합의 크기보다 +1로 크기를 만들어야함.

print(x)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [72]:
len(x[0])

69

In [73]:
len(final)

68

In [3]:
kcc = pd.read_excel('../example_kcc_dataset.xlsx', encoding='cp949')

In [4]:
kcc = kcc.fillna('')

In [5]:
kcc['all'] = kcc['name'] + kcc['size'] + kcc['unit']

In [6]:
kcc.head()

,name,size,unit,all
0,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,"가설사무실발주처,감리사무실,현장사무실 ,화장실 등식"
1,가설경비실 (2.4*2.4 기준),,월,가설경비실 (2.4*2.4 기준)월
2,이동식화장실(FRP),,EA,이동식화장실(FRP)EA
3,EGI펜스,,M2,EGI펜스M2
4,스틸방음벽(자주식),,M2,스틸방음벽(자주식)M2


In [7]:
import re

In [8]:
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [9]:
from konlpy.tag import Okt  
okt=Okt()  
token_result = []
for i in kcc['name']:
    token_result.append(okt.morphs(cleanText(i)))

In [25]:
token_result[:30]

[['가설', '사무실'],
 ['가설', '경비실', '2424', '기준'],
 ['이동식', '화장실', 'FRP'],
 ['EGI', '펜스'],
 ['스틸', '방음벽', '자', '주식'],
 ['스틸', '방음벽', '자립', '식'],
 ['투명', '방음벽', '자', '주식'],
 ['투명', '방음벽', '자립', '식'],
 ['PVC', '방음벽', '자', '주식'],
 ['PVC', '방음벽', '자립', '식'],
 ['홀딩', '도어', '설치', '해체'],
 ['출입문'],
 ['현장', '입', '간판'],
 ['인입', '공사'],
 ['옥외', '외부', '공사'],
 ['구', '내', '설비', '공사'],
 ['한', '전인', '입', '공', '사비'],
 ['임시', '전력', '보증', '보험', '증권', '수수료'],
 ['임시', '동력', '공사'],
 ['전기', '안전', '관리', '대행'],
 ['임시', '전력', '공사'],
 ['임시', '소방시설'],
 ['심정', '공사'],
 ['상하', '수도', '요금'],
 ['상하', '수도', '요금'],
 ['상하', '수도', '요금'],
 ['가설', '용수', '시설'],
 ['가설', '용수', '시설'],
 ['레미콘'],
 ['레미콘', '타', '설']]

In [65]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(token_result)]
models = Doc2Vec(documents, vector_size=1000, window=1, min_count=1, workers=-1,seed=42)

In [66]:
documents[0]

TaggedDocument(words=['가설', '사무실'], tags=[0])

In [67]:
models.similar_by_word('사무실')

[('강화', 0.10591033101081848),
 ('철골', 0.10141515731811523),
 ('커튼월', 0.10076594352722168),
 ('자동', 0.09544691443443298),
 ('설', 0.07928658276796341),
 ('판유리', 0.0755479484796524),
 ('b', 0.07535228133201599),
 ('각종', 0.07378940284252167),
 ('조인트', 0.07357394695281982),
 ('BRACING', 0.06855989992618561)]

In [68]:
ivec = models.infer_vector(doc_words=["터",'파기'], steps=20, alpha=0.025)

In [69]:
sims = models.docvecs.most_similar([ivec])
print(sims)

[(33, 0.08429922163486481), (178, 0.08108982443809509), (514, 0.07940462231636047), (515, 0.07690586149692535), (4, 0.07640202343463898), (276, 0.07180686295032501), (412, 0.07172034680843353), (241, 0.06823918223381042), (620, 0.06793144345283508), (470, 0.0676904246211052)]


In [70]:
dap = list(documents[33])[0]

In [71]:
''.join(dap)

'레미콘'

In [13]:
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res
def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt / len(a), r
a = "가설사무실발주처,감리사무실,현장사무실 ,화장실 등식"
b = "가설경비실 (2.4*2.4 기준)월"
# 2-gram
r2, word2 = diff_ngram(a, b, 2)
print("2-gram:", r2, word2)
# 3-gram
r3, word3  = diff_ngram(a, b, 3)
print("3-gram:", r3, word3)

2-gram: 0.1111111111111111 ['가설', '실 ', '실 ']
3-gram: 0.0 []


In [14]:
import math, sys
from konlpy.tag import Twitter

In [18]:
def spt(text):
    results = []
    twitter = Twitter()
    # 단어의 기본형 사용
    malist = twitter.pos(text, norm=True, stem=True)
    for word in malist:
        # 어미/조사/구두점 등은 대상에서 제외 
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            results.append(word[0])
    return results

In [20]:
spt( kcc['name'][0])

['가설', '사무실']

In [23]:
re = []
for i in kcc['name']:
    re.append(spt(i))

In [24]:
re


[['가설', '사무실'],
 ['가설', '경비실', '2.4', '2.4', '기준'],
 ['이동식', '화장실', 'FRP'],
 ['EGI', '펜스'],
 ['스틸', '방음벽', '자주', '식'],
 ['스틸', '방음벽', '자립', '식'],
 ['투명', '방음벽', '자주', '식'],
 ['투명', '방음벽', '자립', '식'],
 ['PVC', '방음벽', '자주', '식'],
 ['PVC', '방음벽', '자립', '식'],
 ['홀딩', '도어', '설치', '해체'],
 ['출입문'],
 ['현장', '입', '간판'],
 ['인입', '공사'],
 ['옥외', '외부', '공사'],
 ['구', '내', '설비', '공사'],
 ['한', '전인', '입', '공', '사비'],
 ['임시', '전력', '보증', '보험', '증권', '수수료'],
 ['임시', '동력', '공사'],
 ['전기', '안전', '관리', '대행'],
 ['임시', '전력', '공사'],
 ['임시', '소방시설'],
 ['심정', '공사'],
 ['상', '하수도', '요금'],
 ['상', '하수도', '요금'],
 ['상', '하수도', '요금'],
 ['가설', '용수', '시설'],
 ['가설', '용수', '시설'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['철근', 'SD', '400'],
 ['철근', '가공', '조립'],
 ['형틀'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['와이어', '메쉬'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['와이어', '메쉬'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['철근', 'SD', '400'],
 ['철근', '조립'],
 ['형틀'],
 ['레미콘'],
 ['레미콘', '타', '설'],
 ['철근'],
 ['철근', '조립'],
 ['형틀'],
 ['T', 